In [67]:
from ryanair import Ryanair
from datetime import datetime, timedelta
from ryanair import Ryanair
from ryanair.types import Flight
import pprint
from tqdm import tqdm
from datetime import datetime, timedelta
from termcolor import colored
import json
import firebase_admin
from  firebase_admin  import  credentials,  firestore


# Initalivierung 
- Erstmal import 
- dann eine Client instnz initialisiern 

In [6]:
api = Ryanair("EUR")


In [40]:
allDestinationAirports = ["BLQ", "BRI", "BRU", "BUD", "CFU", "CTA", "DUB", "EDI", "EMA", "LPA", "LUX", "MAD", "MAN", "MRS", "OPO", "OTP", "PMI", "PMO", "RAK", "SKG", "SOF", "TGD", "TLL", "TLV", "VCE", "VLC", "VNO"]
departureAirport = "BER"

In [25]:
#TODO Get a list of all Fidays and Saturdays in the next 6 months, and a seperate list for sunday dates ,
#  so that we can iterate over them and get the available flights for each of friday to sunday dates or saturday to sunday dates

        


In [54]:


def generate_weekend_trips(start_date):
    weekend_trips = []
    end_date = start_date + timedelta(days=180)  # 6 months from the start date

    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() == 4:  # Check if the current day is Friday
            saturday = current_date + timedelta(days=1)
            sunday = current_date + timedelta(days=2)
            
            if sunday <= end_date:  # Ensure Sunday is within the 6 month period
                weekend = {
                    "friday": current_date.strftime("%Y-%m-%d"),
                    "saturday": saturday.strftime("%Y-%m-%d"),
                    "sunday": sunday.strftime("%Y-%m-%d")
                }
                weekend_trips.append(weekend)
        
        current_date += timedelta(days=1)

    return weekend_trips


In [55]:
# Generate weekend trips
today = datetime.now().date()
weekend_trip_combinations = generate_weekend_trips(today)

In [56]:
print(weekend_trip_combinations[0])

{'friday': '2024-03-01', 'saturday': '2024-03-02', 'sunday': '2024-03-03'}


In [50]:
len(weekend_trip_combinations)*len(allDestinationAirports)

675

In [35]:
class Flight:
    def __init__(self, departureTime, flightNumber, price, currency, origin, originFull, destination, destinationFull):
        self.departureTime = departureTime
        self.flightNumber = flightNumber
        self.price = price
        self.currency = currency
        self.origin = origin
        self.originFull = originFull
        self.destination = destination
        self.destinationFull = destinationFull

class Trip:
    def __init__(self, totalPrice, outbound, inbound):
        self.totalPrice = totalPrice
        self.outbound = outbound
        self.inbound = inbound


In [61]:
def flight_to_dict(flight):
    return {
        "departureTime": flight.departureTime.isoformat(),
        "flightNumber": flight.flightNumber,
        "price": flight.price,
        "currency": flight.currency,
        "origin": flight.origin,
        "originFull": flight.originFull,
        "destination": flight.destination,
        "destinationFull": flight.destinationFull
    }

# Convert Trip object to JSON serializable dictionary
def trip_to_dict(trip):
    return {
        "totalPrice": trip.outbound.price + trip.inbound.price ,
        "outbound": flight_to_dict(trip.outbound),
        "inbound": flight_to_dict(trip.inbound)
    }

# Function to get and store flight data
def get_and_store_flight_data(weekend_trip_combinations, allDestinationAirports):
    results = {}
    for destination in tqdm(allDestinationAirports, desc="Checking destinations"):
        destination_results = []
        for weekend in tqdm(weekend_trip_combinations, desc=f"Flights to {destination}", leave=False):
            # Mock API call - replace with your actual API call
            trips = api.get_cheapest_return_flights(source_airport=departureAirport, date_from=weekend["friday"], date_to=weekend["saturday"], return_date_from=weekend['saturday'], return_date_to=weekend['sunday'], destination_airport=destination)

            # Handle both single trip object and list of trips
            if not isinstance(trips, list):
                trips = [trips]  # Convert single trip object to a list for uniform processing

            for trip in trips:
                # Print results with colored output
                destination_str = colored(destination, 'yellow')
                departure_str = colored(f"Departure: {weekend['friday']}", 'cyan')
                return_str = colored(f"Return: {weekend['sunday']}", 'magenta')
                result_str = f"Weekend Trip from {departureAirport} to {destination_str}: {departure_str}, {return_str}"
                print(result_str)

                # Convert the trip object to a JSON serializable format and store it
                destination_results.append(trip_to_dict(trip))

        results[destination] = destination_results

    # Save the results to a JSON file
    with open('flights_data.json', 'w') as file:
        json.dump(results, file, indent=4)

    return results


In [63]:


# Call the function and store the results
flights_data = get_and_store_flight_data(weekend_trip_combinations, allDestinationAirports)

print("Data saved to flights_data.json")

Checking destinations:   0%|          | 0/27 [00:00<?, ?it/s]

Weekend Trip from BER to BLQ: Departure: 2024-03-01, Return: 2024-03-03
Weekend Trip from BER to BLQ: Departure: 2024-03-08, Return: 2024-03-10
Weekend Trip from BER to BLQ: Departure: 2024-03-15, Return: 2024-03-17


Weekend Trip from BER to BLQ: Departure: 2024-03-22, Return: 2024-03-24
Weekend Trip from BER to BLQ: Departure: 2024-03-29, Return: 2024-03-31
Weekend Trip from BER to BLQ: Departure: 2024-04-05, Return: 2024-04-07


Weekend Trip from BER to BLQ: Departure: 2024-04-12, Return: 2024-04-14
Weekend Trip from BER to BLQ: Departure: 2024-04-19, Return: 2024-04-21
Weekend Trip from BER to BLQ: Departure: 2024-04-26, Return: 2024-04-28


Weekend Trip from BER to BLQ: Departure: 2024-05-03, Return: 2024-05-05
Weekend Trip from BER to BLQ: Departure: 2024-05-10, Return: 2024-05-12
Weekend Trip from BER to BLQ: Departure: 2024-05-17, Return: 2024-05-19


...

Weekend Trip from BER to VNO: Departure: 2024-06-07, Return: 2024-06-09
Weekend Trip from BER to VNO: Departure: 2024-06-14, Return: 2024-06-16
Weekend Trip from BER to VNO: Departure: 2024-06-21, Return: 2024-06-23


Weekend Trip from BER to VNO: Departure: 2024-06-28, Return: 2024-06-30
Weekend Trip from BER to VNO: Departure: 2024-07-05, Return: 2024-07-07
Weekend Trip from BER to VNO: Departure: 2024-07-12, Return: 2024-07-14


Weekend Trip from BER to VNO: Departure: 2024-07-19, Return: 2024-07-21
Weekend Trip from BER to VNO: Departure: 2024-07-26, Return: 2024-07-28
Weekend Trip from BER to VNO: Departure: 2024-08-02, Return: 2024-08-04


Checking destinations: 100%|██████████| 27/27 [01:20<00:00,  2.98s/it]

Weekend Trip from BER to VNO: Departure: 2024-08-09, Return: 2024-08-11
Weekend Trip from BER to VNO: Departure: 2024-08-16, Return: 2024-08-18
Data saved to flights_data.json


In [64]:
destinationKeys = list(flights_data.keys())
print(destinationKeys)




['BLQ', 'BRI', 'BRU', 'BUD', 'CFU', 'CTA', 'DUB', 'EDI', 'EMA', 'LPA', 'LUX', 'MAD', 'MAN', 'MRS', 'OPO', 'OTP', 'PMI', 'PMO', 'RAK', 'SKG', 'SOF', 'TGD', 'TLL', 'TLV', 'VCE', 'VLC', 'VNO']


In [68]:
cred = credentials.Certificate('/home/rene/Downloads/ryanair-4cd15-firebase-adminsdk-kdyqz-85041d7186.json')
firebase_admin.initialize_app(cred)

In [69]:
db = firestore.client()


In [70]:
#TODO Pase the flights to the database schema and save them in firebase 
for airport_code, flights in flights_data.items():
    # Reference to the airport document in the 'allFlights' collection
    airport_ref = db.collection('allFlights').document(airport_code)

    # Check if the airport document exists
    if not airport_ref.get().exists:
        airport_ref.set({})  # Create a document with the airport code if it does not exist

    # Reference to the 'flightsInfo' subcollection
    flights_info_ref = airport_ref.collection('flightsInfo')

    # Add each flight to the 'flightsInfo' subcollection
    for flight in flights:
        flights_info_ref.add(flight)
        pprint.pprint(f"Added flight to {airport_code} to the database")



'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BLQ to the database'
'Added flight to BRI to the database'
'Added fligh